# Artificial Neural Network for prediciting Camber and Toe extrema

## Install any missing libraries

In [160]:
!pip3 install openpyxl

## Import all libraries

In [1]:
import glob 
import random
import numpy as np
import pandas as pd
import math
import os
import copy
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

## Importing, Cleaning, and Preparing Data

In [2]:
files=glob.glob("Population/*.xlsx",recursive=True)
population=[]
for i in range(0, len(files)):
    parent=pd.read_excel(files[i],header=None).values
    population.append(parent)
population=np.array(population,dtype=object)

In [3]:
all_geometry = []
all_results = []
for i in range(0, len(population)):
    pop = pd.DataFrame(population[i])
    pop = pop.drop(index=[20, 21], axis=0)
    geometry = pop.iloc[14:22, 1:4].values
    results = pop.iloc[82:96, 1:6].values
    all_geometry.append(geometry)
    all_results.append(results)
all_geometry = np.array(all_geometry)
all_results = np.array(all_results)

In [4]:
X_flat=all_geometry.reshape((all_geometry.shape[0], -1))

In [5]:
X=[]
output=[]
for i in range(X_flat.shape[0]):
    for j in range(all_results[i].shape[0]):
        X.append(list(np.append(X_flat[i],all_results[i][j][0])))
        output.append(list(all_results[i][j][1:]))
X=np.array(X)
output=np.array(output)

In [4]:
#output=np.empty(shape=[all_results.shape[0],4])
#for i in range(all_results.shape[0]):
    #output[i,0]=max(all_results[i,:,0])
    #output[i,1]=min(all_results[i,:,0])
    #output[i,2]=max(all_results[i,:,1])
    #output[i,3]=min(all_results[i,:,1])

In [7]:
y=output
X=X.astype('float32')
y=y.astype('float32')

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)

train_df=pd.DataFrame(X_train)
train_df[25]=y_train[:,0]
train_df[26]=y_train[:,1]
train_df[27]=y_train[:,2]
train_df[28]=y_train[:,3]
target=train_df[24:28]
train_df

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,3940.0,-181.000000,485.000000,4259.0,-181.000000,455.000000,4080.000000,-464.500000,255.000000,4070.500000,...,-490.500000,311.000000,4050.500000,-208.0,522.000000,0.0,0.0000,0.0000,6.3974,3.8141
1,3931.5,-226.899994,483.820007,4269.0,-221.199997,478.589996,4072.159912,-526.799988,261.450012,3941.379883,...,-544.099976,341.380005,3961.800049,-243.5,551.599976,60.0,-0.8777,-2.7826,3.6064,-0.6355
2,3942.5,-236.000000,495.420013,4259.0,-237.800003,465.000000,4072.129883,-530.500000,265.350006,3942.500000,...,-546.900024,341.239990,3961.500000,-248.0,572.000000,160.0,-0.5602,3.4410,6.4421,5.7677
3,3931.5,-233.899994,484.820007,4259.0,-234.800003,468.589996,4074.159912,-526.500000,261.450012,3938.500000,...,-542.099976,344.239990,3963.500000,-241.5,564.799988,160.0,-0.8524,-1.5471,4.5875,3.3905
4,3939.0,-180.000000,485.000000,4259.0,-180.000000,455.000000,4082.000000,-464.500000,265.000000,4072.500000,...,-490.500000,321.000000,4050.500000,-208.0,522.000000,160.0,-0.7563,-0.4646,7.4271,4.5231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,3969.0,-180.000000,475.000000,4219.0,-180.000000,465.000000,4082.000000,-463.500000,255.000000,4042.500000,...,-490.500000,311.000000,4050.500000,-218.0,522.000000,40.0,0.1484,0.3249,5.6698,3.3519
368,3931.5,-236.800003,495.820007,4239.5,-238.100006,469.000000,4069.120117,-530.580017,269.350006,3942.500000,...,-544.900024,339.420013,3969.500000,-244.0,569.690002,40.0,0.0974,0.8530,3.2720,3.5824
369,3939.0,-180.000000,485.000000,4259.0,-180.000000,455.000000,4082.000000,-464.500000,255.000000,4072.500000,...,-490.500000,311.000000,4050.500000,-208.0,522.000000,40.0,-0.0093,-0.0994,6.1435,3.8156
370,3939.0,-180.000000,485.000000,4259.0,-180.000000,455.000000,4082.000000,-464.500000,255.000000,4072.500000,...,-490.500000,311.000000,4055.500000,-208.0,522.000000,200.0,-1.0619,-0.6835,7.8543,4.7991


## Scaling Data between 0 and 1

In [8]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_train = scaler.fit_transform(train_df)

# Print out the adjustment that the scaler applied to the data
print("Note: Median values were scaled by multiplying by {:.10f} and adding {:.6f}".format(scaler.scale_[28], scaler.min_[28]))
multiplied_by = scaler.scale_[28]
added = scaler.min_[28]

scaled_train_df = pd.DataFrame(scaled_train, columns=train_df.columns.values)
scaled_train_df

Note: Median values were scaled by multiplying by 0.0407901853 and adding 0.238794


,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,0.523809,0.101491,0.638569,0.800003,0.966159,0.500000,0.816849,0.985294,0.582242,0.985714,...,0.660634,0.162920,0.947369,0.651163,0.166667,0.230769,0.220187,0.757152,0.430563,0.394372
1,0.487013,0.018007,0.600894,1.000000,0.285956,0.893167,0.457886,0.069118,0.770015,0.063427,...,0.054299,0.657869,0.013683,0.238372,0.660000,0.461538,0.181664,0.727969,0.354341,0.212872
2,0.534632,0.001455,0.971265,0.800003,0.005076,0.666667,0.456497,0.014706,0.883552,0.071428,...,0.022624,0.655588,0.010525,0.186047,1.000000,0.846154,0.195600,0.793239,0.431784,0.474059
3,0.487013,0.005275,0.632822,0.800003,0.055837,0.726500,0.549454,0.073529,0.770015,0.042856,...,0.076923,0.704464,0.031578,0.261628,0.880000,0.846154,0.182775,0.740927,0.381135,0.377093
4,0.519482,0.103310,0.638569,0.800003,0.983079,0.500000,0.908432,0.985294,0.873362,1.000000,...,0.660634,0.325839,0.947369,0.651163,0.166667,0.846154,0.186993,0.752279,0.458684,0.423292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,0.649351,0.103310,0.319284,0.000000,0.983079,0.666667,0.908432,1.000000,0.582242,0.785713,...,0.660634,0.162920,0.947369,0.534884,0.166667,0.384615,0.226701,0.760559,0.410692,0.375519
368,0.487013,0.000000,0.984035,0.410004,0.000000,0.733334,0.318695,0.013529,1.000000,0.071428,...,0.045249,0.625937,0.094738,0.232558,0.961500,0.384615,0.224462,0.766097,0.345209,0.384921
369,0.519482,0.103310,0.638569,0.800003,0.983079,0.500000,0.908432,0.985294,0.582242,1.000000,...,0.660634,0.162920,0.947369,0.651163,0.166667,0.384615,0.219779,0.756109,0.423629,0.394433
370,0.519482,0.103310,0.638569,0.800003,0.983079,0.500000,0.908432,0.985294,0.582242,1.000000,...,0.660634,0.162920,1.000000,0.651163,0.166667,1.000000,0.173579,0.749983,0.470351,0.434550


## ANN Model

In [9]:
model = Sequential()

model.add(Dense(50, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(4))

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [10]:
X = scaled_train_df[scaled_train_df.columns[:25]].values
Y = scaled_train_df[scaled_train_df.columns[25:29]].values

In [12]:
model.fit(
    X,
    Y,
    epochs=1000,
    batch_size=10,
    shuffle=False,
    verbose=1
)

Epoch 1/1000
38/38 [==============================] - 0s 1ms/step - loss: 0.0371 - accuracy: 0.8656
Epoch 2/1000
38/38 [==============================] - 0s 1ms/step - loss: 0.0083 - accuracy: 0.9758
Epoch 3/1000
38/38 [==============================] - 0s 1ms/step - loss: 0.0068 - accuracy: 0.9758
Epoch 4/1000
38/38 [==============================] - 0s 1ms/step - loss: 0.0061 - accuracy: 0.9758
Epoch 5/1000
38/38 [==============================] - 0s 1ms/step - loss: 0.0056 - accuracy: 0.9758
Epoch 6/1000
38/38 [==============================] - 0s 1ms/step - loss: 0.0052 - accuracy: 0.9758
Epoch 7/1000
38/38 [==============================] - 0s 1ms/step - loss: 0.0050 - accuracy: 0.9785
Epoch 8/1000
38/38 [==============================] - 0s 1ms/step - loss: 0.0048 - accuracy: 0.9785
Epoch 9/1000
38/38 [==============================] - 0s 1ms/step - loss: 0.0046 - accuracy: 0.9785
Epoch 10/1000
38/38 [==============================] - 0s 1ms/step - loss: 0.0045 - accuracy: 0.9785

38/38 [==============================] - 0s 2ms/step - loss: 2.8934e-04 - accuracy: 0.9973
Epoch 81/1000
38/38 [==============================] - 0s 2ms/step - loss: 2.8499e-04 - accuracy: 0.9973
Epoch 82/1000
38/38 [==============================] - 0s 2ms/step - loss: 2.7433e-04 - accuracy: 0.9973
Epoch 83/1000
38/38 [==============================] - 0s 1ms/step - loss: 2.6868e-04 - accuracy: 0.9919
Epoch 84/1000
38/38 [==============================] - 0s 1ms/step - loss: 2.6861e-04 - accuracy: 0.9973
Epoch 85/1000
38/38 [==============================] - 0s 1ms/step - loss: 2.7006e-04 - accuracy: 0.9973
Epoch 86/1000
38/38 [==============================] - 0s 1ms/step - loss: 2.6825e-04 - accuracy: 0.9973
Epoch 87/1000
38/38 [==============================] - 0s 1ms/step - loss: 2.5768e-04 - accuracy: 0.9973
Epoch 88/1000
38/38 [==============================] - 0s 2ms/step - loss: 2.5603e-04 - accuracy: 0.9973
Epoch 89/1000
38/38 [==============================] - 0s 2ms/step - 

38/38 [==============================] - 0s 1ms/step - loss: 1.6039e-04 - accuracy: 0.9946
Epoch 158/1000
38/38 [==============================] - 0s 1ms/step - loss: 1.1194e-04 - accuracy: 0.9946
Epoch 159/1000
38/38 [==============================] - 0s 1ms/step - loss: 8.5893e-05 - accuracy: 0.9973
Epoch 160/1000
38/38 [==============================] - 0s 1ms/step - loss: 7.2007e-05 - accuracy: 0.9946
Epoch 161/1000
38/38 [==============================] - 0s 1ms/step - loss: 6.8027e-05 - accuracy: 0.9973
Epoch 162/1000
38/38 [==============================] - 0s 1ms/step - loss: 6.4603e-05 - accuracy: 0.9973
Epoch 163/1000
38/38 [==============================] - 0s 1ms/step - loss: 5.8985e-05 - accuracy: 0.9973
Epoch 164/1000
38/38 [==============================] - 0s 1ms/step - loss: 6.1246e-05 - accuracy: 0.9973
Epoch 165/1000
38/38 [==============================] - 0s 1ms/step - loss: 6.8267e-05 - accuracy: 0.9973
Epoch 166/1000
38/38 [==============================] - 0s 1m

38/38 [==============================] - 0s 2ms/step - loss: 3.5461e-04 - accuracy: 0.9946
Epoch 235/1000
38/38 [==============================] - 0s 2ms/step - loss: 3.0465e-04 - accuracy: 0.9919
Epoch 236/1000
38/38 [==============================] - 0s 2ms/step - loss: 2.4987e-04 - accuracy: 0.9946
Epoch 237/1000
38/38 [==============================] - 0s 2ms/step - loss: 2.4698e-04 - accuracy: 0.9919
Epoch 238/1000
38/38 [==============================] - 0s 2ms/step - loss: 2.3843e-04 - accuracy: 0.9946
Epoch 239/1000
38/38 [==============================] - 0s 2ms/step - loss: 2.1790e-04 - accuracy: 0.9946
Epoch 240/1000
38/38 [==============================] - 0s 2ms/step - loss: 1.7591e-04 - accuracy: 0.9946
Epoch 241/1000
38/38 [==============================] - 0s 2ms/step - loss: 1.3954e-04 - accuracy: 0.9946
Epoch 242/1000
38/38 [==============================] - 0s 2ms/step - loss: 1.6993e-04 - accuracy: 0.9919
Epoch 243/1000
38/38 [==============================] - 0s 2m

38/38 [==============================] - 0s 2ms/step - loss: 9.1895e-04 - accuracy: 0.9839
Epoch 312/1000
38/38 [==============================] - 0s 2ms/step - loss: 1.7841e-04 - accuracy: 0.9839
Epoch 313/1000
38/38 [==============================] - 0s 2ms/step - loss: 1.9199e-04 - accuracy: 0.9946
Epoch 314/1000
38/38 [==============================] - 0s 2ms/step - loss: 1.2223e-04 - accuracy: 0.9973
Epoch 315/1000
38/38 [==============================] - 0s 2ms/step - loss: 8.7643e-05 - accuracy: 0.9946
Epoch 316/1000
38/38 [==============================] - 0s 2ms/step - loss: 6.9356e-05 - accuracy: 0.9946
Epoch 317/1000
38/38 [==============================] - 0s 1ms/step - loss: 5.4563e-05 - accuracy: 0.9919
Epoch 318/1000
38/38 [==============================] - 0s 1ms/step - loss: 4.5613e-05 - accuracy: 0.9919
Epoch 319/1000
38/38 [==============================] - 0s 1ms/step - loss: 4.0794e-05 - accuracy: 0.9946
Epoch 320/1000
38/38 [==============================] - 0s 1m

...

38/38 [==============================] - 0s 1ms/step - loss: 6.7057e-05 - accuracy: 0.9973
Epoch 620/1000
38/38 [==============================] - 0s 1ms/step - loss: 4.6144e-05 - accuracy: 0.9973
Epoch 621/1000
38/38 [==============================] - 0s 1ms/step - loss: 4.9880e-05 - accuracy: 0.9973
Epoch 622/1000
38/38 [==============================] - 0s 1ms/step - loss: 4.2865e-05 - accuracy: 0.9973
Epoch 623/1000
38/38 [==============================] - 0s 1ms/step - loss: 3.6293e-05 - accuracy: 0.9973
Epoch 624/1000
38/38 [==============================] - 0s 1ms/step - loss: 2.8882e-05 - accuracy: 0.9973
Epoch 625/1000
38/38 [==============================] - 0s 1ms/step - loss: 2.5202e-05 - accuracy: 0.9973
Epoch 626/1000
38/38 [==============================] - 0s 1ms/step - loss: 2.3382e-05 - accuracy: 1.0000
Epoch 627/1000
38/38 [==============================] - 0s 1ms/step - loss: 2.2622e-05 - accuracy: 1.0000
Epoch 628/1000
38/38 [==============================] - 0s 1m

38/38 [==============================] - 0s 1ms/step - loss: 1.5129e-05 - accuracy: 0.9973
Epoch 697/1000
38/38 [==============================] - 0s 1ms/step - loss: 1.4791e-05 - accuracy: 1.0000
Epoch 698/1000
38/38 [==============================] - 0s 1ms/step - loss: 1.4503e-05 - accuracy: 0.9973
Epoch 699/1000
38/38 [==============================] - 0s 1ms/step - loss: 1.4128e-05 - accuracy: 0.9973
Epoch 700/1000
38/38 [==============================] - 0s 1ms/step - loss: 1.3846e-05 - accuracy: 1.0000
Epoch 701/1000
38/38 [==============================] - 0s 1ms/step - loss: 1.3707e-05 - accuracy: 1.0000
Epoch 702/1000
38/38 [==============================] - 0s 1ms/step - loss: 1.3264e-05 - accuracy: 1.0000
Epoch 703/1000
38/38 [==============================] - 0s 1ms/step - loss: 1.3202e-05 - accuracy: 1.0000
Epoch 704/1000
38/38 [==============================] - 0s 1ms/step - loss: 1.3018e-05 - accuracy: 0.9973
Epoch 705/1000
38/38 [==============================] - 0s 1m

38/38 [==============================] - 0s 1ms/step - loss: 6.8254e-05 - accuracy: 0.9946
Epoch 774/1000
38/38 [==============================] - 0s 1ms/step - loss: 1.3181e-04 - accuracy: 0.9946
Epoch 775/1000
38/38 [==============================] - 0s 1ms/step - loss: 1.4629e-04 - accuracy: 0.9973
Epoch 776/1000
38/38 [==============================] - 0s 1ms/step - loss: 1.5242e-04 - accuracy: 0.9919
Epoch 777/1000
38/38 [==============================] - 0s 1ms/step - loss: 1.5223e-04 - accuracy: 0.9919
Epoch 778/1000
38/38 [==============================] - 0s 1ms/step - loss: 8.5554e-05 - accuracy: 0.9973
Epoch 779/1000
38/38 [==============================] - 0s 1ms/step - loss: 5.3302e-05 - accuracy: 0.9973
Epoch 780/1000
38/38 [==============================] - 0s 1ms/step - loss: 3.4520e-05 - accuracy: 0.9946
Epoch 781/1000
38/38 [==============================] - 0s 1ms/step - loss: 3.1235e-05 - accuracy: 0.9946
Epoch 782/1000
38/38 [==============================] - 0s 1m

38/38 [==============================] - 0s 1ms/step - loss: 1.9149e-04 - accuracy: 0.9919
Epoch 851/1000
38/38 [==============================] - 0s 1ms/step - loss: 8.7852e-05 - accuracy: 1.0000
Epoch 852/1000
38/38 [==============================] - 0s 1ms/step - loss: 1.6962e-04 - accuracy: 0.9919
Epoch 853/1000
38/38 [==============================] - 0s 1ms/step - loss: 5.8139e-05 - accuracy: 0.9973
Epoch 854/1000
38/38 [==============================] - 0s 1ms/step - loss: 9.7352e-05 - accuracy: 0.9946
Epoch 855/1000
38/38 [==============================] - 0s 1ms/step - loss: 4.7787e-05 - accuracy: 1.0000
Epoch 856/1000
38/38 [==============================] - 0s 1ms/step - loss: 7.7660e-05 - accuracy: 0.9973
Epoch 857/1000
38/38 [==============================] - 0s 1ms/step - loss: 4.0448e-05 - accuracy: 0.9973
Epoch 858/1000
38/38 [==============================] - 0s 1ms/step - loss: 5.3510e-05 - accuracy: 0.9973
Epoch 859/1000
38/38 [==============================] - 0s 1m

38/38 [==============================] - 0s 1ms/step - loss: 3.6354e-05 - accuracy: 0.9946
Epoch 928/1000
38/38 [==============================] - 0s 1ms/step - loss: 3.1355e-05 - accuracy: 0.9946
Epoch 929/1000
38/38 [==============================] - 0s 1ms/step - loss: 3.4147e-05 - accuracy: 0.9946
Epoch 930/1000
38/38 [==============================] - 0s 2ms/step - loss: 6.7902e-05 - accuracy: 0.9973
Epoch 931/1000
38/38 [==============================] - 0s 1ms/step - loss: 8.6034e-05 - accuracy: 0.9973
Epoch 932/1000
38/38 [==============================] - 0s 1ms/step - loss: 1.2872e-04 - accuracy: 0.9946
Epoch 933/1000
38/38 [==============================] - 0s 1ms/step - loss: 3.2951e-04 - accuracy: 0.9919
Epoch 934/1000
38/38 [==============================] - 0s 1ms/step - loss: 7.6001e-04 - accuracy: 0.9866
Epoch 935/1000
38/38 [==============================] - 0s 1ms/step - loss: 5.9842e-04 - accuracy: 0.9812
Epoch 936/1000
38/38 [==============================] - 0s 1m

## Predictions

In [13]:
prediction = model.predict(X[:5])
print('Prediction with scaling - ',format(prediction))

y_pred = prediction
y_pred -= added
y_pred /= multiplied_by
print("Prediction - ",format(y_pred))

Prediction with scaling -  [[0.22315449 0.7595993  0.40999132 0.3954531 ]
 [0.17913875 0.722125   0.3586457  0.20674752]
 [0.1935194  0.79436773 0.42245382 0.47893095]
 [0.17911023 0.7424777  0.3770985  0.37962246]
 [0.18672577 0.7543971  0.45661688 0.42599607]]
Prediction -  [[-0.38341108 12.76791     4.197025    3.8406103 ]
 [-1.4624877  11.8492      2.9382513  -0.7856394 ]
 [-1.109936   13.620282    4.502552    5.8871284 ]
 [-1.463187   12.348162    3.3906343   3.452511  ]
 [-1.2764868  12.640373    5.340083    4.5893927 ]]


In [14]:
Y[:5]

array([[0.22018716, 0.75715166, 0.4305631 , 0.39437175],
       [0.18166417, 0.7279693 , 0.35434103, 0.21287173],
       [0.19559951, 0.793239  , 0.43178385, 0.47405946],
       [0.1827746 , 0.74092656, 0.3811348 , 0.37709302],
       [0.18699253, 0.7522792 , 0.45868415, 0.42329198]], dtype=float32)

In [ ]:
y_pred=model.predict(X_)